# 2.2 Predicting RNA secondary structure

In [3]:
import pandas as pd
import numpy as np

# Try to import the library we just installed
try:
    import RNA
    print(f"✅ ViennaRNA version {RNA.__version__} is ready.")
except ImportError:
    print("❌ ViennaRNA not found. (If you installed it, restart your kernel).")

✅ ViennaRNA version 2.7.1 is ready.


In [4]:
# ---------------------------------------------------------
# 1. SETUP
# ---------------------------------------------------------
input_file = 'merged_data.xlsx'
# We will save the result here
output_file = 'merged_data_2.2.xlsx' 

# ---------------------------------------------------------
# 2. FOLDING FUNCTION
# ---------------------------------------------------------
def fold_sequence_only(row):
    # Safety check for empty sequences
    if pd.isna(row['Precursor sequence']):
        return pd.Series({'structure': None, 'MFE': None})
    
    # Clean and prepare sequence (remove whitespace, uppercase, T->U)
    seq = str(row['Precursor sequence']).strip().upper().replace('T', 'U')
    
    # --- Perform Folding ---
    # RNA.fold returns (structure_string, mfe_float)
    structure, mfe = RNA.fold(seq)
    
    return pd.Series({
        'structure': structure,
        'MFE': mfe
    })

# ---------------------------------------------------------
# 3. EXECUTION
# ---------------------------------------------------------
try:
    # Read the Excel file (all sheets)
    all_sheets = pd.read_excel(input_file, sheet_name=None)
    processed_sheets = {}

    with pd.ExcelWriter(output_file) as writer:
        for species, df in all_sheets.items():
            print(f"🧬 Folding {species} ({len(df)} sequences)...")
            
            # Apply the folding function
            # This returns a DataFrame with just 'structure' and 'MFE'
            folding_results = df.apply(fold_sequence_only, axis=1)
            
            # Remove existing structure/MFE columns if re-running (to avoid duplicates)
            df_clean = df.drop(columns=['structure', 'MFE'], errors='ignore')
            
            # Combine: Original Data + New Folding Columns
            final_df = pd.concat([df_clean, folding_results], axis=1)
            
            # Save to the new sheet
            final_df.to_excel(writer, sheet_name=species, index=False)
            
    print(f"\n✅ Success! Dataset with folding saved to: {output_file}")

except FileNotFoundError:
    print(f"❌ File not found: {input_file}. Make sure it's in the same folder.")
except Exception as e:
    print(f"❌ An error occurred: {e}")

🧬 Folding Human (514 sequences)...
🧬 Folding House mouse (404 sequences)...
🧬 Folding Fruit fly (140 sequences)...
🧬 Folding Roundworm (138 sequences)...

✅ Success! Dataset with folding saved to: merged_data_2.2.xlsx


# 2.3 Extracting Structural Features

In [6]:
import pandas as pd
import numpy as np
import re

# ---------------------------------------------------------
# CONFIGURATION
# ---------------------------------------------------------
INPUT_FILE = "merged_data_2.2.xlsx"
OUTPUT_FILE = "merged_data_final.xlsx"

# ---------------------------------------------------------
# HELPER FUNCTIONS
# ---------------------------------------------------------

def get_dot_bracket_pairs(structure):
    """
    Parses a dot-bracket string (e.g. "((..))") into a list of pairs (i, j).
    Returns a set of tuples {(i, j), ...} where i < j.
    """
    pairs = set()
    stack = []
    for i, char in enumerate(structure):
        if char == '(':
            stack.append(i)
        elif char == ')':
            if stack:
                start = stack.pop()
                pairs.add((start, i))
    return pairs

def get_consecutive_unmatched_max(sub_structure):
    """
    Returns the length of the longest run of dots '.' in a string.
    """
    if not sub_structure:
        return 0
    # Split by any paired characters ('(' or ')') to get chunks of dots
    # e.g. "((...))..." -> ["", "", "...", "", "...", ""]
    # Then take max length
    tokens = re.split(r'[()]', sub_structure)
    return max(len(t) for t in tokens)

def analyze_structure(row):
    # Safety checks
    if pd.isna(row.get('structure')) or pd.isna(row.get('Precursor sequence')):
        return pd.Series({})
    
    seq = str(row['Precursor sequence']).strip().upper().replace('T', 'U')
    struct = str(row['structure']).strip()
    
    # Check if lengths match (crucial for indexing)
    if len(seq) != len(struct):
        # Fallback: if lengths differ, we can't map pairs accurately. Return empty.
        return pd.Series({'Feature_Extraction_Error': 'Length Mismatch'})

    # 1. Identify Regions (Indices)
    # We need to find where Mature and Star start/end in the Precursor
    mat_seq = str(row.get('Mature sequence', '')).strip().upper().replace('T', 'U')
    star_seq = str(row.get('Star sequence', '')).strip().upper().replace('T', 'U')
    
    # Find start index. Note: find() returns first occurrence.
    mat_start = seq.find(mat_seq)
    mat_end = mat_start + len(mat_seq) if mat_start != -1 else -1
    
    star_start = seq.find(star_seq) if star_seq and star_seq != 'NAN' else -1
    star_end = star_start + len(star_seq) if star_start != -1 else -1

    # 2. Parse Structure
    pairs = get_dot_bracket_pairs(struct)
    
    # -----------------------------------------------------
    # CALCULATE REQUIRED FEATURES 
    # -----------------------------------------------------
    
    # 1. MFE is already in the row
    
    # 2. Number of base-pairs in Mature
    # Interpretation: How many bases IN the mature region are paired?
    # (Checking if index k is in mature range and struct[k] is '(' or ')')
    mat_bp_count = 0
    if mat_start != -1:
        mat_sub = struct[mat_start:mat_end]
        mat_bp_count = mat_sub.count('(') + mat_sub.count(')')
        mat_bp_count = mat_bp_count
        
    # 3. Number of base-pairs in Star
    star_bp_count = 0
    if star_start != -1:
        star_sub = struct[star_start:star_end]
        star_bp_count = (star_sub.count('(') + star_sub.count(')'))

    # 4. Loop Size (Terminal Loop)
    # The loop enclosed by the pair with NO other pairs inside it.
    # If multiple (e.g. multiloop), we take the largest one.
    terminal_loop_size = 0
    candidates = []
    
    for (i, j) in pairs:
        # Check if this pair encloses any other pair
        is_terminal = True
        for (k, l) in pairs:
            if (k > i) and (l < j):
                is_terminal = False
                break
        
        if is_terminal:
            # The size is the number of nucleotides between i and j
            candidates.append(j - i - 1)
            
    if candidates:
        terminal_loop_size = max(candidates)

    # 5, 6, 7. GC / AU / GU counts across Precursor
    gc_count = 0
    au_count = 0
    gu_count = 0
    
    for (i, j) in pairs:
        b1 = seq[i]
        b2 = seq[j]
        pair_str = "".join(sorted([b1, b2])) # Sort to normalize (e.g. UG -> GU)
        
        if pair_str == 'CG':
            gc_count += 1
        elif pair_str == 'AU':
            au_count += 1
        elif pair_str == 'GU':
            gu_count += 1
            
    # -----------------------------------------------------
    # CALCULATE ADDITIONAL FEATURES
    # -----------------------------------------------------
    
    # A. Unmatched nts on mature side
    mat_unmatched = 0
    mat_max_consecutive = 0
    if mat_start != -1:
        mat_sub = struct[mat_start:mat_end]
        mat_unmatched = mat_sub.count('.')
        mat_max_consecutive = get_consecutive_unmatched_max(mat_sub)

    # B. Unmatched nts on star side
    star_unmatched = 0
    star_max_consecutive = 0
    if star_start != -1:
        star_sub = struct[star_start:star_end]
        star_unmatched = star_sub.count('.')
        star_max_consecutive = get_consecutive_unmatched_max(star_sub)
        
    # C. Unmatched nts on precursor (Total)
    pre_unmatched = struct.count('.')

    return pd.Series({
        # Required
        'Mature_BP_Count': mat_bp_count,
        'Star_BP_Count': star_bp_count,
        'Terminal_Loop_Size': terminal_loop_size,
        'GC_Pairs_Precursor': gc_count,
        'AU_Pairs_Precursor': au_count,
        'GU_Pairs_Precursor': gu_count,
        
        # Additional
        'Mature_Unmatched_Count': mat_unmatched,
        'Star_Unmatched_Count': star_unmatched,
        'Precursor_Unmatched_Count': pre_unmatched,
        'Mature_Max_Consecutive_Unmatched': mat_max_consecutive,
        'Star_Max_Consecutive_Unmatched': star_max_consecutive
    })

# ---------------------------------------------------------
# EXECUTION LOOP
# ---------------------------------------------------------
def main():
    print(f"Reading {INPUT_FILE}...")
    try:
        xls = pd.ExcelFile(INPUT_FILE)
    except FileNotFoundError:
        print(f"Error: {INPUT_FILE} not found.")
        return

    with pd.ExcelWriter(OUTPUT_FILE, engine='openpyxl') as writer:
        for sheet_name in xls.sheet_names:
            print(f"Processing {sheet_name}...")
            df = pd.read_excel(xls, sheet_name=sheet_name)
            
            # Apply analysis
            features_df = df.apply(analyze_structure, axis=1)
            
            # Combine original data with new features
            final_df = pd.concat([df, features_df], axis=1)
            
            # Save
            final_df.to_excel(writer, sheet_name=sheet_name, index=False)
            
    print(f"✅ Done! Features saved to {OUTPUT_FILE}")

if __name__ == "__main__":
    main()

Reading merged_data_2.2.xlsx...
Processing Human...
Processing House mouse...
Processing Fruit fly...
Processing Roundworm...
✅ Done! Features saved to merged_data_final.xlsx
